## Import Libraries

In [1]:
from binance.client import Client
import pandas as pd
from datetime import datetime, timedelta
import time
import pytz

## Set up Client

In [2]:
# No API key needed for public market data (historical candles)
client = Client()

## Define a function to fetch historical klines

In [3]:
client = Client()

In [32]:
def get_klines_batched(symbol, interval, start_str, end_str=None, timezone=None, sleep_time=0.5):
    """
    Fetch historical OHLCV data in batches from Binance (no 1000-limit issues).

    Parameters:
    - symbol: str, e.g., 'BTCUSDT'
    - interval: str, e.g., '1m', '1h', '1d'
    - start_str: str, start date like '2023-01-01'
    - end_str: str or None, end date like '2025-01-01'
    - timezone: str or None, e.g., 'America/Mexico_City'
    - sleep_time: float, seconds to sleep between batch requests

    Returns:
    - DataFrame with OHLCV data and optional timezone localization
    """
    interval_map = {
        '1m': timedelta(minutes=1),
        '3m': timedelta(minutes=3),
        '5m': timedelta(minutes=5),
        '15m': timedelta(minutes=15),
        '30m': timedelta(minutes=30),
        '1h': timedelta(hours=1),
        '2h': timedelta(hours=2),
        '4h': timedelta(hours=4),
        '6h': timedelta(hours=6),
        '8h': timedelta(hours=8),
        '12h': timedelta(hours=12),
        '1d': timedelta(days=1),
        '3d': timedelta(days=3),
        '1w': timedelta(weeks=1),
        '1M': timedelta(days=30)  # approx.
    }

    df_total = pd.DataFrame()
    start_ts = int(pd.to_datetime(start_str).timestamp() * 1000)
    end_ts = int(pd.to_datetime(end_str).timestamp() * 1000) if end_str else None
    delta = interval_map[interval] * 1000  # delta in ms

    while True:
        klines = client.get_historical_klines(symbol=symbol,
                                              interval=interval,
                                              start_str=start_ts,
                                              end_str=end_ts,
                                              limit=1000)
        
        if not klines:
            break

        temp_df = pd.DataFrame(klines, columns=[
            "timestamp", "open", "high", "low", "close", "volume",
            "close_time", "quote_asset_volume", "number_of_trades",
            "taker_buy_base_volume", "taker_buy_quote_volume", "ignore"
        ])
        
        temp_df["timestamp"] = pd.to_datetime(temp_df["timestamp"], unit="ms")
        df_total = pd.concat([df_total, temp_df], ignore_index=True)

        # Break if last candle exceeds end_ts or no more data
        last_ts = klines[-1][0]
        if end_ts and last_ts >= end_ts or len(klines) < 1000:
            break
        
        start_ts = last_ts + int(delta.total_seconds() * 1000)
        time.sleep(sleep_time)

    df_total.set_index("timestamp", inplace=True)

    if timezone is not None:
        df_total.index = df_total.index.tz_localize("UTC").tz_convert(timezone)

    df_total = df_total[["open", "high", "low", "close", "volume"]].astype(float)
    return df_total

In [30]:
def get_klines_batched_combined(symbol, interval, start_str, end_str, timezone=None, sleep_time=0.5, recent_pad_days=10):
    """
    Smart wrapper: fetch historical + recent data from Binance to avoid cutoffs.

    Parameters:
    - recent_pad_days: int, how many days to reserve for recent fetch

    Returns:
    - Complete DataFrame with full date range
    """
    # Step 1: define split point for "recent" section
    end_dt = pd.to_datetime(end_str)
    recent_start_dt = end_dt - timedelta(days=recent_pad_days)

    # 1️⃣ Fetch full historical (up to recent_start_dt)
    historical_df = get_klines_batched(
        symbol=symbol,
        interval=interval,
        start_str=start_str,
        end_str=recent_start_dt.strftime("%Y-%m-%d"),
        timezone=timezone,
        sleep_time=sleep_time
    )

    # 2️⃣ Fetch recent (last few days)
    recent_df = get_klines_batched(
        symbol=symbol,
        interval=interval,
        start_str=recent_start_dt.strftime("%Y-%m-%d"),
        end_str=end_str,
        timezone=timezone,
        sleep_time=sleep_time
    )

    # Merge & deduplicate
    full_df = pd.concat([historical_df, recent_df])
    full_df = full_df[~full_df.index.duplicated(keep='last')]

    return full_df.sort_index()

In [4]:
def get_klines(symbol, interval, start_str, end_str=None, timezone=None):
    """
    Fetches historical OHLCV data from Binance and optionally localizes time.

    Parameters:
    - symbol: str, e.g., 'BTCUSDT'
    - interval: str, e.g., '1h', '1d'
    - start_str: str, start date like '2023-01-01'
    - end_str: str or None, end date like '2025-01-01'
    - timezone: str or None, e.g., 'America/Mexico_City'

    Returns:
    - DataFrame with localized datetime index (if timezone is provided)
    """
    klines = client.get_historical_klines(symbol=symbol,
                                          interval=interval,
                                          start_str=start_str,
                                          end_str=end_str)

    df = pd.DataFrame(klines, columns=[
        "timestamp", "open", "high", "low", "close", "volume",
        "close_time", "quote_asset_volume", "number_of_trades",
        "taker_buy_base_volume", "taker_buy_quote_volume", "ignore"
    ])
    
    # Convert timestamp to datetime (UTC)
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")
    df.set_index("timestamp", inplace=True)

    # Localize if timezone is specified
    if timezone is not None:
        df.index = df.index.tz_localize("UTC").tz_convert(timezone)

    df = df[["open", "high", "low", "close", "volume"]].astype(float)
    return df


## Get Data

In [7]:
btc = get_klines(
    symbol="BTCUSDT",
    interval="1h",
    start_str="2023-01-01",
    end_str="2025-06-29",
    timezone="America/Mexico_City"
)

In [8]:
sol = get_klines(
    symbol="SOLUSDT",
    interval="1h",
    start_str="2023-01-01",
    end_str="2025-06-29",
    timezone="America/Mexico_City"
)

In [10]:
eth = get_klines(
    symbol="ETHUSDT",
    interval="1h",
    start_str="2023-01-01",
    end_str="2025-06-29",
    timezone="America/Mexico_City"
)

## Store data into SQLite3 DB (step by step)

In [13]:
import sqlite3

In [14]:
db_path = "data/crypto_data.sqlite"

### Confirm data is ready

#### BTCUSDT

In [46]:
btc.head()

,open,high,low,close,volume
timestamp,,,,,
2022-12-31 18:00:00-06:00,16541.77,16545.70,16508.39,16529.67,4364.83570
2022-12-31 19:00:00-06:00,16529.59,16556.80,16525.78,16551.47,3590.06669
2022-12-31 20:00:00-06:00,16551.47,16559.77,16538.14,16548.19,3318.84038
2022-12-31 21:00:00-06:00,16548.19,16548.19,16518.21,16533.04,4242.08050
2022-12-31 22:00:00-06:00,16533.04,16535.97,16511.92,16521.85,4285.00909


#### Reset the index and prepare the data

In [47]:
btc_to_insert = btc.reset_index()
btc_to_insert["timestamp"] = btc_to_insert["timestamp"].astype(str)  # Convert to string for SQLite

#### Insert into SQLite database

In [49]:
with sqlite3.connect(db_path) as conn:
    btc_to_insert.to_sql("btc_1h", conn, if_exists="replace", index=False)


# Can change if_exists="replace" to:
# "append" to add new rows (if not duplicates),
# "fail" to raise error if table exists.

#### Confirm insertion

In [50]:
with sqlite3.connect(db_path) as conn:
    count = pd.read_sql("SELECT COUNT(*) FROM btc_1h", conn)
print(count)

   COUNT(*)
0     11241


In [51]:
sqlite3.connect(db_path).close()

## Store data into SQLite3 DB (function)

In [15]:
db_path

'data/crypto_data.sqlite'

### Function

In [16]:
def insert_ohlcv_to_sqlite(df, table_name, db_path):
    df_to_insert = df.reset_index()
    df_to_insert["timestamp"] = df_to_insert["timestamp"].astype(str)
    with sqlite3.connect(db_path) as conn:
        df_to_insert.to_sql(table_name, conn, if_exists="replace", index=False)

### ETH

In [17]:
insert_ohlcv_to_sqlite(
    df=eth,
    table_name="eth_1h",
    db_path=db_path
    )

### SOL

In [18]:
insert_ohlcv_to_sqlite(
    df=sol,
    table_name="sol_1h",
    db_path=db_path
    )

### BTC

In [19]:
insert_ohlcv_to_sqlite(
    df=btc,
    table_name="btc_1h",
    db_path=db_path
    )